## Read Data

In [147]:
import pandas as pd
import numpy as np
import pingouin as pg

# Load the dataset
df = pd.read_csv("output.csv")

# Display the first few rows of the dataset

df.drop('date', axis=1, inplace=True)
df.head()

resultsDf = pd.DataFrame()

In [17]:
df = pd.DataFrame([
    { 'user_id':1, 'ranking': 1, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':1, 'ranking': 2, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':1, 'ranking': 3, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':1, 'ranking': 4, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':1, 'ranking': 5, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':1, 'ranking': 6, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':1, 'ranking': 7, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':1, 'ranking': 8, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':1, 'ranking': 9, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':1, 'ranking': 10, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':1, 'ranking': 1, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 2, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':1, 'ranking': 3, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 4, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 5, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':1, 'ranking': 6, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 7, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':1, 'ranking': 8, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 9, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':1, 'ranking': 10, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 1, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':2, 'ranking': 2, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':2, 'ranking': 3, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':2, 'ranking': 4, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':2, 'ranking': 5, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':2, 'ranking': 6, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':2, 'ranking': 7, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':2, 'ranking': 8, 'algorithm': 'A', 'rating': 1 },
    { 'user_id':2, 'ranking': 9, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':2, 'ranking': 10, 'algorithm': 'A', 'rating': 5 },
    { 'user_id':2, 'ranking': 1, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 2, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':2, 'ranking': 3, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 4, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 5, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':2, 'ranking': 6, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 7, 'algorithm': 'B', 'rating': 5 },
    { 'user_id':2, 'ranking': 8, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 9, 'algorithm': 'B', 'rating': 1 },
    { 'user_id':2, 'ranking': 10, 'algorithm': 'B', 'rating': 1 }
])

## MRR Metric

In [141]:
def compute_mrr(data, threshold=4.0):
    # Filter out only relevant items
    relevant_data = data[data['rating'] >= threshold]
    
    # Get the rank of the first relevant item for each user
    min_relevant_rank = relevant_data.groupby(['user_id', 'algorithm'])['ranking'].min().reset_index()
    
    # Compute the reciprocal rank for each user
    min_relevant_rank['reciprocal_rank'] = 1.0 / min_relevant_rank['ranking']
    
    # Compute the mean of the reciprocal ranks
    mrr = min_relevant_rank.groupby('algorithm')['reciprocal_rank'].mean()
    
    return mrr

# Compute MRR for the data
mrr_values = compute_mrr(df)
resultsDf['MRR'] = mrr_values

## MAP Metric

In [142]:
def compute_average_precision(dataframe, k=10, relevant_threshold=4):
    dataframe = dataframe.sort_values(by=["user_id", "algorithm", "ranking"])
    
    dataframe["is_relevant"] = dataframe["rating"] >= relevant_threshold

    dataframe["precision_at_k"] = dataframe.groupby(["user_id", "algorithm"])["is_relevant"].cumsum() / (dataframe["ranking"].astype(int))
    
    dataframe = dataframe[dataframe["ranking"] <= k]

    avg_precision = dataframe[dataframe["is_relevant"] == True].groupby(["user_id", "algorithm"])["precision_at_k"].mean()

    return avg_precision.groupby(level="algorithm").mean()

map_per_algorithm = compute_average_precision(df)
print(map_per_algorithm)
resultsDf['MAP'] = map_per_algorithm

algorithm
B1                0.728485
B1-SCSA_PLUS      0.777916
CS                0.695136
CS-SCSA_PLUS      0.669751
SC                0.793744
SC-SCSA_PLUS      0.786361
SCSA              0.717515
SCSA-SCSA_PLUS    0.735007
Name: precision_at_k, dtype: float64


## NDCG Metric

In [143]:
def compute_ndcg_at_k(df, k=10):
    # Sort dataframe by ranking (predicted ranking)
    df = df.sort_values(by=["user_id", "algorithm", "ranking"])
    
    # Get top k items
    df = df.head(k)
    
    # Compute the relevance of each item (1 if rating >= 4 else 0)
    relevance = (df['rating'] >= 4).astype(int).values
    
    # Compute DCG
    dcg = np.sum(relevance / np.log2(np.arange(2, k+2)))
    
    # Compute IDCG (ideal DCG)
    idcg = np.sum(sorted(relevance, reverse=True) / np.log2(np.arange(2, k+2)))
    
    # Compute NDCG
    return dcg / idcg if idcg != 0 else 0

# Group by user and algorithm and compute NDCG@10
ndcg_scores = df.groupby(['user_id', 'algorithm']).apply(compute_ndcg_at_k)

# Average NDCG@10 for each algorithm
avg_ndcg_scores = ndcg_scores.groupby('algorithm').mean()
resultsDf['NDCG'] = avg_ndcg_scores

## Precision Metric

In [144]:
def compute_precision_at_k(ratings, k):
    relevant_items = sum(1 for rating in ratings[:k] if rating >= 4)
    return relevant_items / k

grouped = df.groupby(['user_id', 'algorithm']).apply(lambda x: x.sort_values('ranking')['rating'].tolist())

# Compute Precision@10 for each group
precision_at_10_values = grouped.apply(lambda x: compute_precision_at_k(x, 10))

# Compute average Precision@10 for each algorithm
precision_at_10_per_algorithm = precision_at_10_values.groupby('algorithm').mean()

resultsDf['Precision'] = precision_at_10_per_algorithm

## Annova

In [19]:
from scipy import stats

t_val, p_val = stats.ttest_ind(mrr_values, avg_ndcg_scores)
print("T-value:", t_val)
print("P-value:", p_val)

T-value: -0.47506595255348427
P-value: 0.6420661451287575


## Test T-Student

In [115]:
resultsTransposed = resultsDf.transpose()
print(resultsTransposed)

algorithm        B1  B1-SCSA_PLUS        CS  CS-SCSA_PLUS        SC  \
MRR        0.748028      0.793284  0.732232      0.641063  0.792398   
MAP        0.728485      0.777916  0.695136      0.669751  0.793744   
NDCG       0.753666      0.788356  0.731911      0.702362  0.795544   

algorithm  SC-SCSA_PLUS      SCSA  SCSA-SCSA_PLUS  
MRR            0.778407  0.735235        0.756254  
MAP            0.786361  0.717515        0.735007  
NDCG           0.792450  0.738417        0.753788  


In [116]:
print(resultsTransposed['B1']['MRR'])

0.7480277185501065


In [37]:
from scipy import stats

t_val, p_val = stats.ttest_rel(resultsDf.iloc[:,0], resultsDf.iloc[:,1])
print("T-value:", t_val)
print("P-value:", p_val)

t_val, p_val = stats.ttest_rel(resultsDf.iloc[:,1], resultsDf.iloc[:,2])
print("T-value:", t_val)
print("P-value:", p_val)

t_val, p_val = stats.ttest_rel(resultsDf.iloc[:,0], resultsDf.iloc[:,2])
print("T-value:", t_val)
print("P-value:", p_val)



T-value: 1.2502338252923473
P-value: 0.25138748785670967
T-value: -4.34531222446711
P-value: 0.0033741069395569817
T-value: -1.3069701975016905
P-value: 0.2325176043356958


In [117]:
variances = resultsTransposed.var(axis=1)  # Compute variance for each row (algorithm-metric combination)
print(variances)

nan_inf_counts = resultsTransposed.isin([np.nan, np.inf, -np.inf]).sum()
print(nan_inf_counts)


MRR     0.002408
MAP     0.002008
NDCG    0.001101
dtype: float64
algorithm
B1                0
B1-SCSA_PLUS      0
CS                0
CS-SCSA_PLUS      0
SC                0
SC-SCSA_PLUS      0
SCSA              0
SCSA-SCSA_PLUS    0
dtype: int64


In [126]:
t_stat, p_val = stats.ttest_rel(resultsTransposed['SCSA'],resultsTransposed['SCSA-SCSA_PLUS'])
print(t_stat, p_val)

-10.902733929590394 0.008307888934725258


In [145]:
algorithms = ['SC', 'B1', 'CS', 'SCSA', 'SCSA-SCSA_PLUS', 'B1-SCSA_PLUS', 'CS-SCSA_PLUS', 'SC-SCSA_PLUS']
metrics = ['MRR', 'MAP', 'NDCG', 'Precision']
results = []

for metric in metrics:
    for i in range(len(algorithms)):
        for j in range(i+1, len(algorithms)):
            alg1 = algorithms[i]
            alg2 = algorithms[j]
            # Perform t-test
            t_stat, p_val = stats.ttest_rel(resultsTransposed[alg1],resultsTransposed[alg2])

            if(alg2.startswith(f'{alg1}-')):
                results.append({'Metric': metric, 'Comparison': f'{alg1} vs {alg2}', 'p-value': p_val})

#Convert results to a DataFrame
results_df = pd.DataFrame(results)

#3. Adjust for multiple comparisons using the Bonferroni correction
alpha = 0.05
results_df['Significant'] = results_df['p-value'] < alpha

print(results_df[results_df['Significant'] == True])

       Metric              Comparison   p-value  Significant
1         MRR      B1 vs B1-SCSA_PLUS  0.010189         True
3         MRR  SCSA vs SCSA-SCSA_PLUS  0.008308         True
5         MAP      B1 vs B1-SCSA_PLUS  0.010189         True
7         MAP  SCSA vs SCSA-SCSA_PLUS  0.008308         True
9        NDCG      B1 vs B1-SCSA_PLUS  0.010189         True
11       NDCG  SCSA vs SCSA-SCSA_PLUS  0.008308         True
13  Precision      B1 vs B1-SCSA_PLUS  0.010189         True
15  Precision  SCSA vs SCSA-SCSA_PLUS  0.008308         True


In [149]:
t_stat, p_val = pg.anova(resultsTransposed)

for metric in metrics:
    for i in range(len(algorithms)):
        for j in range(i+1, len(algorithms)):
            alg1 = algorithms[i]
            alg2 = algorithms[j]
            # Perform t-test
            t_stat, p_val = pg.anova(resultsTransposed[alg1],resultsTransposed[alg2])

            if(alg2.startswith(f'{alg1}-')):
                results.append({'Metric': metric, 'Comparison': f'{alg1} vs {alg2}', 'p-value': p_val})

#Convert results to a DataFrame
results_df = pd.DataFrame(results)

#3. Adjust for multiple comparisons using the Bonferroni correction
alpha = 0.05
results_df['Significant'] = results_df['p-value'] < alpha

print(results_df[results_df['Significant'] == True])

ValueError: DV and data must be specified